<a href="https://colab.research.google.com/github/KumbhakarM/LabWork/blob/main/Colab_GEE_LST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install geemap and earthengine-api if not already installed
!pip install -q geemap
import geemap
import ee

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.6 MB/s eta 0:00:00


In [2]:
import geemap
import ee


# Authenticate and initialize
ee.Authenticate()
ee.Initialize(project='ee-my-mk')

# Load Patna district from GAUL GADM dataset
districts = ee.FeatureCollection("FAO/GAUL/2015/level2")
patna = districts.filter(ee.Filter.And(
    ee.Filter.eq('ADM1_NAME', 'Bihar'),
    ee.Filter.eq('ADM2_NAME', 'Patna')
))


# Filter Landsat 8/9 images for March 2025
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .merge(ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")) \
    .filterBounds(patna) \
    .filterDate("2025-03-01", "2025-03-31") \
    .filter(ee.Filter.lt('CLOUD_COVER', 20))

# Choose the first image with low cloud cover
image = landsat.sort('CLOUD_COVER').first().clip(patna)

# Apply scaling and composite
def scaleLandsat(img):
    opticalBands = img.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = img.select('ST_B10').multiply(0.00341802).add(149.0)
    return img.addBands(opticalBands, overwrite=True).addBands(thermalBand.rename('BT'))

# Create median composite
image = landsat.map(scaleLandsat).median().clip(patna)

tiles = landsat.aggregate_array("LANDSAT_PRODUCT_ID").getInfo()
print("Tiles used in composite:\n", tiles)


# NDVI for emissivity estimation
ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# Emissivity calculation based on NDVI
def calculateEmissivity(ndvi):
    pv = ndvi.subtract(0.2).divide(0.3).pow(2)  # Proportion of vegetation
    emissivity = pv.multiply(0.004).add(0.986)
    return emissivity.rename('Emissivity')

emissivity = calculateEmissivity(ndvi)

# Corrected LST calculation
bt = image.select('BT')  # Brightness Temperature in Kelvin
epsilon = emissivity

# Constants
gamma = ee.Number(0.00115)
rho = ee.Number(14388)

# Corrected LST formula
lst = bt.divide(
    ee.Image(1).add(
        bt.divide(rho).multiply(emissivity.log()).multiply(gamma)
    )
).subtract(273.15).rename('LST')  # Convert K to °C


# Add NDVI, emissivity, LST to image
image = image.addBands([ndvi, emissivity, lst])

# Visualization parameters
vis_params = {
    'min': 20,
    'max': 40,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'orange', 'red']
}

# Display on map
Map = geemap.Map(center=[25.6, 85.1], zoom=9)
Map.addLayer(image.select('LST'), vis_params, 'LST (°C) - Mar 2025')
Map.addLayer(patna.style(color='black', fillColor='00000000'), {}, 'Patna District')
Map.addLayerControl()
Map


Tiles used in composite:
 ['LC08_L2SP_140042_20250317_20250327_02_T1', 'LC08_L2SP_141042_20250308_20250312_02_T1', 'LC08_L2SP_141042_20250324_20250331_02_T1', 'LC08_L2SP_141043_20250308_20250312_02_T1', 'LC08_L2SP_141043_20250324_20250331_02_T1', 'LC09_L2SP_140042_20250325_20250326_02_T1', 'LC09_L2SP_140043_20250309_20250311_02_T1', 'LC09_L2SP_140043_20250325_20250326_02_T1']


Map(center=[25.6, 85.1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [3]:
# Set export parameters
task = ee.batch.Export.image.toDrive(
    image=lst,  # LST image
    description='LST_Patna_March2025',
    folder='EarthEngine',  # Optional: Folder name in your Google Drive
    fileNamePrefix='LST_Patna_March2025',
    region=patna.geometry(),  # Clip to Patna district
    scale=30,  # Landsat native resolution
    maxPixels=1e13,
    crs='EPSG:4326'  # Optional: Export in WGS84
)

# Start export task
task.start()
print("Export task started. Check https://code.earthengine.google.com/tasks to monitor.")


Export task started. Check https://code.earthengine.google.com/tasks to monitor.
